<a href="https://colab.research.google.com/github/SHPDH09/Blustock-Team-1/blob/main/Data_Quality_Monitoring_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pandas numpy fpdf openpyxl sqlalchemy
!pip install streamlit pyngrok


  Preparing metadata (setup.py) ... done
  Created wheel for fpdf: filename=fpdf-1.7.2-py2.py3-none-any.whl size=40704 sha256=00207e091e38503bcf1d8962806089ec0fdef75a8fd5b8f1442268e903025be5
  Stored in directory: /root/.cache/pip/wheels/65/4f/66/bbda9866da446a72e206d6484cd97381cbc7859a7068541c36
Successfully built fpdf
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 85.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 101.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 7.4 MB/s eta 0:00:00


In [2]:
from google.colab import files
import pandas as pd

uploaded = files.upload()  # Upload sales_data.csv
data = pd.read_csv(list(uploaded.keys())[0])
data.head()


Saving hw_200.csv to hw_200.csv


,Index,"Height(Inches)""","""Weight(Pounds)"""
0,1,65.78,112.99
1,2,71.52,136.49
2,3,69.40,153.03
3,4,68.22,142.34
4,5,67.79,144.30


In [3]:
def check_missing_values(df):
    return df.isnull().sum()

def check_duplicates(df):
    return df.duplicated().sum()

def check_data_types(df):
    return df.dtypes

missing = check_missing_values(data)
duplicates = check_duplicates(data)
print("Missing Values:\n", missing)
print("Duplicates:", duplicates)
print("Data Types:\n", check_data_types(data))


Missing Values:
 Index                0
 Height(Inches)"     0
 "Weight(Pounds)"    0
dtype: int64
Duplicates: 0
Data Types:
 Index                  int64
 Height(Inches)"     float64
 "Weight(Pounds)"    float64
dtype: object


In [4]:
def calculate_quality_score(df):
    total_records = len(df)
    missing = df.isnull().sum().sum()
    duplicates = df.duplicated().sum()

    score = 100
    if missing > 0:
        score -= (missing / (total_records * len(df.columns))) * 50
    if duplicates > 0:
        score -= (duplicates / total_records) * 30
    return max(score, 0)

score = calculate_quality_score(data)
print("Data Quality Score:", score, "%")


Data Quality Score: 100 %


In [5]:
from fpdf import FPDF

def generate_report(missing, duplicates, score):
    pdf = FPDF()
    pdf.add_page()
    pdf.set_font("Arial", size=12)
    pdf.cell(200, 10, txt="Data Quality Report", ln=True, align="C")
    pdf.cell(200, 10, txt=f"Missing Values: {missing.sum()}", ln=True)
    pdf.cell(200, 10, txt=f"Duplicate Records: {duplicates}", ln=True)
    pdf.cell(200, 10, txt=f"Quality Score: {score:.2f}%", ln=True)
    pdf.output("data_quality_report.pdf")

generate_report(missing, duplicates, score)
files.download("data_quality_report.pdf")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [6]:
%%writefile app.py
import streamlit as st
import pandas as pd

st.title("Data Quality Monitoring Dashboard")

data = pd.read_csv("sales_data.csv")
missing = data.isnull().sum().sum()
duplicates = data.duplicated().sum()
score = 100 - ((missing / (len(data)*len(data.columns))) * 50) - ((duplicates/len(data))*30)

st.metric("Missing Values", missing)
st.metric("Duplicate Records", duplicates)
st.metric("Quality Score", f"{score:.2f}%")


Writing app.py
